# Practice Querying

In [22]:
# imports
import matplotlib.pyplot as plt 
from snowexsql.db import get_db
from snowexsql.data import SiteData, ImageData, LayerData, PointData
from snowexsql.conversions import query_to_geopandas
import datetime

In [2]:
# load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)
print('snowexsql database successfully loaded')

snowexsql database successfully loaded


In [ ]:
# Great for debugging especially when trying different queries
# session.rollback()

## Snow Pits --> _LayerData_ & _SiteData_ in the snowexsql database
### Example 1: 
Let's find all the pits that overlap with UAVSAR data

1a). Unsure of the fly over date, but know which instrument you'd like to overlap with, here's how:

In [11]:
# Form on the Images table that returns Raster collection dates
qry = session.query(ImageData.date)

# Filter for UAVSAR data
qry = qry.filter(ImageData.surveyors == 'UAVSAR team, JPL')

# Grab the unique dates
qry = qry.distinct()
qry
# Execute the query 
dates = qry.all() 

# Clean up the dates to remove the tuple
dates = [d[0] for d in dates] 
print('UAVSAR flight dates are: ', dates)

# Find all the snow pits done on these days and return a geopandas df
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(dates))
df = query_to_geopandas(qry, engine)
df.head()

# Close your session to avoid hanging transactions
session.close()

UAVSAR flight dates are:  [datetime.date(2020, 1, 31), datetime.date(2020, 2, 12)]


1b). Want to select from a range of dates near the fly over date? Here's how:

In [20]:
# Pick a day from the list of dates
dt = dates[0] 

# Find all the snow pits done on these days and return a geopandas df
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date == dt)
df_exact = query_to_geopandas(qry, engine)
df_exact.head()

,geom,site_id,date
0,POINT (740652.000 4327445.000),2C2,2020-01-31
1,POINT (744396.000 4323540.000),8C26,2020-01-31
2,POINT (741960.000 4326644.000),6C10,2020-01-31
3,POINT (741493.000 4326833.000),1C8,2020-01-31
4,POINT (745340.000 4322754.000),8S28,2020-01-31


In [21]:
# Form a date range to query on either side of our chosen day 
date_range = [dt + i * datetime.timedelta(days=1) for i in [-1, 0, 1]]

# Find all the snow pits done on these days and return a geopandas df
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(date_range))
df_range = query_to_geopandas(qry, engine)
df_range

,geom,site_id,date
0,POINT (744561.000 4322721.000),5S21,2020-02-01
1,POINT (746228.000 4322671.000),9S39,2020-02-01
2,POINT (745937.000 4322754.000),2S37,2020-02-01
3,POINT (746546.000 4324066.000),8N45,2020-02-01
4,POINT (746350.000 4321976.000),6C34,2020-02-01
5,POINT (746149.000 4322488.000),3S38,2020-02-01
6,POINT (743652.000 4322680.000),3S14,2020-02-01
7,POINT (747055.000 4323916.000),5N50,2020-02-01
8,POINT (746553.000 4323759.000),6N46,2020-02-01
9,POINT (746303.000 4322571.000),9S40,2020-02-01


1c). Have a known date in mind that you wish to select, here's how:

In [ ]:
# Find all the data that was collected on 2-12-2020
dt = datetime.date(2020, 2, 12)

# Grab all Point data instruments from our date
point_instruments = session.query(PointData.instrument).filter(PointData.date == dt).distinct().all()
# print('Point Data are: ', point_instruments)

# Grab all Layer data instruments from our date
layer_instruments = session.query(LayerData.instrument, LayerData.type).filter(LayerData.date == dt).distinct().all()
# print(layer_instruments)

# Grab all Image data instruments from our date
image_instruments = session.query(ImageData.instrument).filter(ImageData.date == dt).distinct().all()
# print(image_instruments)

qry = session.query(PointData.geom, PointData.instrument, PointData.site_id, PointData.date)
qry = qry.filter(PointData.date == dt) 
df = query_to_geopandas(qry, engine)
df

In [ ]:
dt = datetime.date(2020, 2, 12)
dt

### Your turn
#### try this: query for a different airborne instrument (hint: airborne lidar--> 'ASO Inc.')

In [ ]:
#

In [ ]:
#RANDOM QUESTIONS/ CODE EXAMPLE BOX
qry = session.query(ImageData.surveyors) # can I print a list of surveyor options??
surveyors_lst = session.query(ImageData.surveyors).distinct().all()
surveyors_lst
# qry = session.query(SiteData.site_id, SiteData.date)
# qry = qry.filter(SiteData.date.in_(dates))
# results = qry.all()

### Example 2: 
grab all pits based on flight line pattern

1). only want to analyize pits from the 'N' flight line?

2). what about the 'N' flight line for a 1 week period?

In [ ]:
# Grab density


session.rollback()

q = session.query(LayerData).filter(LayerData.type == 'density').filter(LayerData.site_id.contains('N')).limit(100)
df = query_to_geopandas(q, engine)
# Convert density to float
df['value'] = df['value'].astype(float)

df 

df.plot()
plt.show()


# Calculate SWE
# swe_lambda = lambda row: row['value'] * (row['depth'] - row['bottom_depth']) / 100
# df['swe'] = df.apply(swe_lambda, axis=1)

In [ ]:
df

### Example 3: 
grab all pits based on the depth and vegetation matrix

1). interested in isolating shallow pits? here's how: 

2). what about pits in dense canopy cover? 